# LLM Integrations

Track and analyze your OpenAI API calls with AgentOps

## Setting Up API Keys

Before using OpenAI with AgentOps, you need to set up your API keys. You can obtain:

- **OPENAI_API_KEY**: From the [OpenAI Platform](https://platform.openai.com/api-keys)
- **AGENTOPS_API_KEY**: From your [AgentOps Dashboard](https://app.agentops.ai/)
    
Then to set them up, you can either export them as environment variables or set them in a `.env` file.

```
OPENAI_API_KEY="your_openai_api_key_here"
AGENTOPS_API_KEY="your_agentops_api_key_here"
```

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Set up environment variables with fallback values
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AGENTOPS_API_KEY"] = os.getenv("AGENTOPS_API_KEY")

## Usage
Initialize AgentOps at the beginning of your application to automatically track all OpenAI API calls:

![img](imgs/9.llm.png)

In [2]:
import agentops
from openai import OpenAI
      
# Initialize AgentOps
agentops.init()

# Create OpenAI client
client = OpenAI()

# Make API calls as usual - AgentOps will track them automatically
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
)

print(response.choices[0].message.content)

🖇 AgentOps: You're on the agentops free plan 🤔
🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'


The capital of France is Paris.


Streaming

In [2]:
import agentops
from openai import OpenAI

# Initialize AgentOps
agentops.init()

# Create OpenAI client
client = OpenAI()

# Make a streaming API call
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short poem about AI."}
    ],
    stream=True
)

# Process the streaming response
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

🖇 AgentOps: You're on the agentops free plan 🤔
🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'


In circuits woven, thoughts reside,  
A dance of numbers, side by side.  
From depths of data, wisdom streams,  
In silent algorithms, we chase our dreams.  

A mirror held to humankind,  
With every question, answers find.  
In code and logic, sparks ignite,  
A spark of meaning in the night.  

Yet as we build with hand and heart,  
Remember well, we're not apart.  
For in this realm where circuits play,  
We shape the dawn of a new day.  

IndexError: list index out of range

Function Calling

In [2]:
import json
import agentops
from openai import OpenAI

# Initialize AgentOps
agentops.init()

# Create OpenAI client
client = OpenAI()

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

# Function implementation
def get_weather(location):
    return json.dumps({"location": location, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]})

# Make a function call API request
messages = [
    {"role": "system", "content": "You are a helpful weather assistant."},
    {"role": "user", "content": "What's the weather like in Boston?"}
]

response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

# Process response
response_message = response.choices[0].message
messages.append({"role": "assistant", "content": response_message.content, "tool_calls": response_message.tool_calls})

# Execute tool and update result
if response_message.tool_calls:
    # Process each tool call
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name == "get_weather":
            function_response = get_weather(function_args.get("location"))
            
            # Add tool response to messages
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response,
                }
            )
    
    # Get a new response from the model
    second_response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
    )
    
    print(second_response.choices[0].message.content)
else:
    print(response_message.content)

🖇 AgentOps: You're on the agentops free plan 🤔
🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'


The current weather in Boston, MA is sunny and windy with a temperature of 72 degrees Fahrenheit.


## Examples

- [OpenAI Sync Example](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai/openai_example_sync.ipynb)
- [OpenAI Async Example](https://docs.agentops.ai/v2/integrations/openai)
- [Web Search Example](https://github.com/AgentOps-AI/agentops/blob/main/examples/openai/web_search.ipynb)
- [Multi-Tool Orchestration](https://docs.agentops.ai/v2/examples/openai)